# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [3]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [4]:
#document_tree = ET.parse( './data/mondial_database_less.xml' )

In [5]:
# print names of all countries
#for child in document_tree.getroot():
    #print (child.find('name').text)

In [6]:
# print names of all countries and their cities
#for element in document_tree.iterfind('country'):
    #print '* ' + element.find('name').text + ':',
    #capitals_string = ''
    #for subelement in element.getiterator('city'):
        #capitals_string += subelement.find('name').text + ', '
    #print capitals_string[:-2]

****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [7]:
document = ET.parse( './data/mondial_database.xml' )

In [9]:
root = document.getroot()

In [15]:
import pandas as pd
#10 countries with the lowest infant mortality rates
n1 =[]
inf_mor=[]
for country in root.findall('country'):
    name = country.find('name').text
   
    for infant in country.getiterator('infant_mortality'):
        infant_mortality = (float(infant.text))
        inf_mor.append(infant_mortality)
        if infant_mortality != None:
             n1.append(name)
d=[n1, inf_mor]  
df = pd.DataFrame(d)
df = df.transpose()
df.columns = ['Name', 'infant_mortality']
df = df.sort_values(by='infant_mortality',ascending=1)
df.head(10)

,Name,infant_mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.6
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [42]:
#10 cities with largest population
import pandas as pd
d1=[]
n2 = []
p1 = []
for element in root.iterfind('country'):
    for subelement in element.getiterator('city'):
        name = subelement.find('name').text
        
        pop = ''
        for child in subelement.getiterator('population'):           
            
            pop = (float(child.text))
        p1.append(pop)
        if pop != None:
            n2.append(name)            
d1 = [n2,p1]
#Making dataframe using lists of population and name
df1 = pd.DataFrame(d1)
df1 = df1.transpose()
df1.columns= ['name', 'population']
df1 = df1[df1['population'] != '']
df1 = df1.sort_values(by='population',ascending=0)
df1.head(10)

,name,population
1341,Shanghai,2.23155e+07
771,Istanbul,1.37105e+07
1527,Mumbai,1.24424e+07
479,Moskva,1.19795e+07
1340,Beijing,1.17166e+07
2810,São Paulo,1.11523e+07
1342,Tianjin,1.10903e+07
1064,Guangzhou,1.10714e+07
1582,Delhi,1.10346e+07
1067,Shenzhen,1.03584e+07


In [498]:
#10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
import pandas as pd
eg = []
pop_new = []
prcnt = []
for element in document.iterfind('country'):
    name = element.find('name').text
    for subelement in element.iterfind('population'):        
        pop = int(float(subelement.text))    
    ethnicgroup=''
    for subelement in element.iterfind('ethnicgroup'): 
        ethnicgroup = subelement.text
        eg.append(ethnicgroup)
        percent = float(subelement.get('percentage'))
        prcnt.append(percent)
        pop_new1 = pop * percent * 0.01        
        pop_new.append(pop_new1)
df3 = [eg,pop_new]
df3 = pd.DataFrame(df3)
df3 = df3.transpose()
df3.columns = ['ethnic_group', 'population']
df3.groupby('ethnic_group').sum().sort_values(by='population', ascending=False).head(10)


,population
ethnic_group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [43]:
#name and country of a) longest river
import pandas as pd

name = []
code = []
for country in document.iterfind('country'):
    name1 = country.find('name').text
    code1 = country.attrib['car_code']
    name.append(name1)
    code.append(code1)
dcon=[name, code]
dfcon= pd.DataFrame(dcon)
dfcon = dfcon.transpose()
dfcon.columns = (['country', 'country_code'])

river_list=[]    
for river in document.iterfind('river'):
    river_name = river.find('name').text
    river_length = river.find('length')
    con = river.attrib['country']
    if river_length != None:
        river_length = river.find('length').text
    river_list.append([con, river_name, river_length])
dfriv = pd.DataFrame(river_list)
dfriv.columns = (['country_code', 'river_name', 'river_length'])
dfriv['river_length']= pd.to_numeric(dfriv.river_length)
dfriv = dfriv.sort_values(by= 'river_length', ascending=False)
s = dfriv['country_code'].str.split(' ').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'country_code'
del dfriv['country_code']
dfriv = dfriv.join(s)
longriv = dfriv.merge(dfcon, how='outer')
longriv = longriv.sort_values(by= 'river_length', ascending=False).head(10)
longriv

,river_name,river_length,country_code,country
310,Amazonas,6448.0,PE,Peru
299,Amazonas,6448.0,CO,Colombia
302,Amazonas,6448.0,BR,Brazil
226,Jangtse,6380.0,CN,China
225,Hwangho,4845.0,CN,China
43,Lena,4400.0,R,Russia
364,Zaire,4374.0,ZRE,Zaire
390,Zaire,4374.0,RCB,Congo
227,Mekong,4350.0,CN,China
246,Mekong,4350.0,LAO,Laos


In [44]:
#name and country of  b) largest lake 
import pandas as pd
name = []
code = []
for country in document.iterfind('country'):
    name1 = country.find('name').text
    code1 = country.attrib['car_code']
    name.append(name1)
    code.append(code1)
dcon=[name, code]
dfcon= pd.DataFrame(dcon)
dfcon = dfcon.transpose()
dfcon.columns = (['country', 'country_code'])

lake_list=[]    
for lake in document.iterfind('lake'):
    lake_name = lake.find('name').text
    lake_area = lake.find('area')
    con = lake.attrib['country']
    if lake_area != None:
        lake_area = lake.find('area').text
    lake_list.append([con, lake_name, lake_area])
dflak = pd.DataFrame(lake_list)
dflak.columns = (['country_code', 'lake_name', 'lake_area'])
dflak['lake_area']= pd.to_numeric(dflak.lake_area)
dflak = dflak.sort_values(by= 'lake_area', ascending=False)
s = dflak['country_code'].str.split(' ').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'country_code'
del dflak['country_code']
dflak = dflak.join(s)

larglak= dflak.merge(dfcon, how='outer')
larglak = larglak.sort_values(by= 'lake_area', ascending=False).head(10)
larglak


,lake_name,lake_area,country_code,country
56,Caspian Sea,386400.0,R,Russia
72,Caspian Sea,386400.0,IR,Iran
73,Caspian Sea,386400.0,AZ,Azerbaijan
74,Caspian Sea,386400.0,KAZ,Kazakhstan
77,Caspian Sea,386400.0,TM,Turkmenistan
148,Lake Superior,82103.0,CDN,Canada
154,Lake Superior,82103.0,USA,United States
116,Lake Victoria,68870.0,EAT,Tanzania
104,Lake Victoria,68870.0,EAK,Kenya
108,Lake Victoria,68870.0,EAU,Uganda


In [546]:
#name and country of c) airport at highest elevation
import pandas as pd
name = []
code = []
for country in document.iterfind('country'):
    name1 = country.find('name').text
    code1 = country.attrib['car_code']
    name.append(name1)
    code.append(code1)
dcon=[name, code]
dfcon= pd.DataFrame(dcon)
dfcon = dfcon.transpose()
dfcon.columns = (['country', 'country_code'])

arprt_list=[]    
for arprt in document.iterfind('airport'):
    arprt_name = arprt.find('name').text
    arprt_elev = arprt.find('elevation')
    con = arprt.attrib['country']
    if arprt_elev != None:
        arprt_elev = arprt.find('elevation').text
    arprt_list.append([con, arprt_name, arprt_elev])
dfarprt = pd.DataFrame(arprt_list)
dfarprt.columns = (['country_code', 'arprt_name', 'arprt_elev'])
dfarprt['arprt_elev']= pd.to_numeric(dfarprt.arprt_elev)
dfarprt = dfarprt.sort_values(by= 'arprt_elev', ascending=False)


higharprt= dfarprt.merge(dfcon, how='outer')
higharprt = higharprt.sort_values(by= 'arprt_elev', ascending=False).head(10)
higharprt

,country_code,arprt_name,arprt_elev,country
0,BOL,El Alto Intl,4063.0,Bolivia
7,CN,Lhasa-Gonggar,4005.0,China
8,CN,Yushu Batang,3963.0,China
104,PE,Juliaca,3827.0,Peru
105,PE,Teniente Alejandro Velasco Astete Intl,3311.0,Peru
1,BOL,Juana Azurduy De Padilla,2905.0,Bolivia
120,EC,Mariscal Sucre Intl,2813.0,Ecuador
106,PE,Coronel Fap Alfredo Mendivil Duarte,2719.0,Peru
107,PE,Mayor General FAP Armando Revoredo Iglesias Ai...,2677.0,Peru
124,MEX,Licenciado Adolfo Lopez Mateos Intl,2581.0,Mexico


In [2]:
#for child in document.getroot():
    #print (child.tag)
    #for kid in child:
        #print(kid.tag)